In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os, sys
from IPython.display import display
from IPython.display import Image as _Imgdis
from PIL import Image
import numpy as np
from time import time
from time import sleep
from scipy import ndimage
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import re
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

Using TensorFlow backend.


In [2]:
os.listdir(path='.')
listOfFile = os.listdir("./gender")
x=len(listOfFile)
for i in range(0,x):
    print(listOfFile[i])
    
onlyfiles=[]
x=len(listOfFile)
for i in range(0,x):
    folder="./gender/"+listOfFile[i]
    print(folder)
    onlyfiles2 = [f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]
    for t in range(0, len(onlyfiles2)):
        onlyfiles.append(folder+"/"+onlyfiles2[t])
print("Working with {0} images".format(len(onlyfiles)))
print(onlyfiles[5])

e
k
./gender/e
./gender/k
Working with 10079 images
./gender/e/001 (103).jpg


In [3]:
from scipy import ndimage
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import re

train_files = []
y_train = []
i=0
for _file in onlyfiles:
    train_files.append(_file)
    r1 = re.findall("\/.\/", _file)
    mystring = str(r1)
    r2 = re.findall("k|e", mystring)
    y_train.append(r2)
    i=i+1
    
print("Files in train_files: %d" % len(train_files))

# Original Dimensions
image_width = 256
image_height = 256
ratio = 4

image_width = int(image_width / ratio)
image_height = int(image_height / ratio)

channels = 3
nb_classes = 1

dataset = np.ndarray(shape=(len(train_files), channels, image_height, image_width),
                     dtype=np.float32)

i = 0
for _file in train_files:
    img = load_img(_file)
    img = img.resize((256,256))
    img.thumbnail((image_width, image_height))
    # Convert to Numpy Array
    x = img_to_array(img)  
    x = x.reshape((3, 64, 64))
    # Normalize
    #x = (x - 128.0) / 128.0
    dataset[i] = x
    i += 1
    if i % 3000 == 0:
        print("%d images to array" % i)
print("All images to array!")

Files in train_files: 10079
3000 images to array
6000 images to array
9000 images to array
All images to array!


In [20]:
#np.save('y_trainknn.npy', y_train)
#np.save('datasetknn.npy', dataset)
dataset=np.load('datasetknn.npy')
y_train=np.load('y_trainknn.npy')

In [27]:
df=pd.DataFrame(y_train)

In [28]:
df.head()

,0
0,e
1,e
2,e
3,e
4,e


In [31]:
#df[0]=df[0].astype('category')
df.dtypes

0    category
dtype: object

In [32]:
y_train=df.values.tolist()

In [37]:
# Normalize the data
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
y_train = df[0]
#label_encoder = LabelEncoder()
#vec = label_encoder.fit_transform(y_train)
#y_train = to_categorical(y_train)
#dataset = dataset / 255.0
#y_train = y_train / 255.0
print("dataset shape: ",dataset.shape)
print("y_train shape: ",y_train.shape)

dataset shape:  (10079, 3, 64, 64)
y_train shape:  (10079,)


In [38]:
X_train, X_test, y_train, y_test = train_test_split(dataset, y_train, test_size=0.2, random_state=33)
print("x train: ",X_train.shape)
print("x test: ",X_test.shape)
print("y train: ",y_train.shape)
print("y test: ",y_test.shape)

x train:  (8063, 3, 64, 64)
x test:  (2016, 3, 64, 64)
y train:  (8063,)
y test:  (2016,)


In [39]:
#reshape data to fit model
x_train = X_train.reshape(8063,64,64,3)
x_test = X_test.reshape(2016,64,64,3)
print("x train: ",x_train.shape)
print("x test: ",x_test.shape)

#y_train = pd.Categorical(y_train).codes
#y_test = pd.Categorical(y_test).codes

x train:  (8063, 64, 64, 3)
x test:  (2016, 64, 64, 3)


In [40]:
number_of_train = x_train.shape[0]
number_of_test = x_test.shape[0]
print(number_of_train,number_of_test)

x_train_flatten = x_train.reshape(number_of_train,x_train.shape[1]*x_train.shape[2]*x_train.shape[3])
x_test_flatten = x_test .reshape(number_of_test,x_test.shape[1]*x_test.shape[2]*x_test.shape[3])
print("X train flatten",x_train_flatten.shape)
print("X test flatten",x_test_flatten.shape)

x_train = x_train_flatten.T
x_test = x_test_flatten.T
y_test = y_test.T
y_train = y_train.T
print("x train: ",x_train.shape)
print("x test: ",x_test.shape)
print("y train: ",y_train.shape)
print("y test: ",y_test.shape)

8063 2016
X train flatten (8063, 12288)
X test flatten (2016, 12288)
x train:  (12288, 8063)
x test:  (12288, 2016)
y train:  (8063,)
y test:  (2016,)


In [41]:

x_train=x_train.T
x_test=x_test.T

In [42]:

print("x train: ",x_train.shape)
print("x test: ",x_test.shape)
print("y train: ",y_train.shape)
print("y test: ",y_test.shape)

x train:  (8063, 12288)
x test:  (2016, 12288)
y train:  (8063,)
y test:  (2016,)


In [43]:
model = KNeighborsClassifier(n_neighbors=3)
# Train the model using the training sets
model.fit(x_train,y_train)
#Predict Output
predicted= model.predict(x_test) 
print(metrics.accuracy_score(y_test, predicted))
#print(predicted)

0.7514880952380952
